<a href="https://colab.research.google.com/github/slipaway/melonplaylist/blob/%EC%9E%AC%ED%98%84/matrix_factorization_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [4]:
pip install implicit

     |████████████████████████████████| 1.1MB 2.7MB/s 
  Created wheel for implicit: filename=implicit-0.4.2-cp36-cp36m-linux_x86_64.whl size=3420085 sha256=77eb8366782864034cd88d132356575f7fc761407a9e98aef9a99fb9fa8e849e
  Stored in directory: /root/.cache/pip/wheels/1b/48/b1/1aebe3acc3afb5589e72d3e7c3ffc3f637dc4721c1a974dff7
Successfully built implicit


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import timedelta, datetime
import glob
import os
import re
import seaborn as sns
import warnings
import scipy
warnings.filterwarnings('ignore')
import implicit

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
folder = '/content/gdrive/My Drive/melon kakao'
filelist = os.chdir(folder)

In [7]:
genre_gn_all=pd.read_json('genre_gn_all.json', typ='series')
train=pd.read_json('train.json')
song_meta=pd.read_json('song_meta.json')
test=pd.read_json('val.json')

In [10]:
plylst_date = train[['updt_date', 'songs']]
plylst_date_unnest = np.dstack(
    (
        np.repeat(plylst_date.updt_date.values, list(map(len, plylst_date.songs))), 
        np.concatenate(plylst_date.songs.values)
    )
)
plylst_date = pd.DataFrame(data=plylst_date_unnest[0], columns = plylst_date.columns)

del plylst_date_unnest

plylst_date["updt_date"] = plylst_date["updt_date"].map(lambda x: x[:4] + x[5:7] + x[8:10])
song_date = song_meta[["id","issue_date"]]
plylst_song_date = pd.merge(plylst_date, song_date, left_on="songs", right_on="id", how='left')
plylst_song_date["issue_date"] = plylst_song_date["issue_date"].astype(str)

plylst_song_date["strange"] = plylst_song_date["updt_date"] < plylst_song_date["issue_date"]
strange_songs = plylst_song_date[plylst_song_date["strange"] == True].drop('songs', axis=1)

strange_songs=strange_songs.drop_duplicates(['id'])
print(strange_songs)

strange_songs_list=strange_songs['id'].values.astype(str)


        updt_date      id issue_date  strange
180      20170915  418694   20180511     True
183      20170915  135950   20180420     True
184      20170915  304687   20180612     True
679      20131017  563802   20161202     True
934      20160510  306385   20180808     True
...           ...     ...        ...      ...
5264610  20180119  123443   20180209     True
5268374  20100513  663564   20100519     True
5274802  20190824  139433   20191011     True
5278046  20140808  486629   20161202     True
5283397  20060407  398250   20060408     True

[2972 rows x 4 columns]


In [11]:
plylst_song_map=train[['id', 'songs']]

#unnest songs

plylst_song_map_unnest=np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))),
        np.concatenate(plylst_song_map.songs.values)
    )
)

#unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map=pd.DataFrame(data=plylst_song_map_unnest[0], columns=plylst_song_map.columns)
plylst_song_map['id']=plylst_song_map['id'].astype(str)
plylst_song_map['songs']=plylst_song_map['songs'].astype(str)

del plylst_song_map_unnest

#플레이리스트에 strange 곡 제거한 후, 수록된 곡의 총 개수(중복 포함)

plylst_song_map_remove=plylst_song_map[np.logical_not(plylst_song_map.songs.isin(strange_songs_list))]
plylst_song_map_remove.shape

#train set에 새로운 column : remove_songs 생성
#remove_songs : strange 곡 제거한 후 다시 만든 플레이리스트

plylst_song_group=plylst_song_map_remove.groupby('id')['songs'].apply(list).reset_index(name='remove_songs')
train['id']=train['id'].astype(str)
train_remove = pd.merge(train, plylst_song_group)

train_remove.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,remove_songs
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,"[525514, 129701, 383374, 562083, 297861, 13954..."
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,"[432406, 675945, 497066, 120377, 389529, 24427..."
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,"[83116, 276692, 166267, 186301, 354465, 256598..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,"[394031, 195524, 540149, 287984, 440773, 10033..."
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,"[159327, 553610, 5130, 645103, 294435, 100657,..."


In [14]:
train_remove['songs']=train_remove['remove_songs']
train_remove=train_remove.drop('remove_songs', axis=1)

In [16]:
test=pd.read_json('val.json')
test_song=test[['id', 'songs']]
filtering=test['songs'].astype('str')=='[]'
test_song=test_song[~filtering]
test_song.info()

#플레이리스트 아이디(id)와 수록곡(songs) 추출
plylst_song_map_train=train_remove[['id', 'songs']]
plylst_song_map_test=test_song
plylst_song_map=pd.concat([plylst_song_map_train, plylst_song_map_test])


#unnest songs

plylst_song_map_unnest=np.dstack(
    (
        np.repeat(plylst_song_map.id.values, list(map(len, plylst_song_map.songs))),
        np.concatenate(plylst_song_map.songs.values)
    )
)

#unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map=pd.DataFrame(data=plylst_song_map_unnest[0], columns=plylst_song_map.columns)
plylst_song_map['id']=plylst_song_map['id'].astype(str)
plylst_song_map['songs']=plylst_song_map['songs'].astype(str)

del plylst_song_map_unnest

#플레이리스트와 song을 행, 열로 가지는 sparse matrix 만들기
plylst_song_map['id']=plylst_song_map['id'].astype('category')
plylst_song_map['songs']=plylst_song_map['songs'].astype('category')

sparse_song=scipy.sparse.coo_matrix((np.ones(plylst_song_map.shape[0]),
                                (plylst_song_map['songs'].cat.codes.copy(),
                                 plylst_song_map['id'].cat.codes.copy())))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18636 entries, 0 to 23014
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18636 non-null  int64 
 1   songs   18636 non-null  object
dtypes: int64(1), object(1)
memory usage: 436.8+ KB


In [17]:
model_song=implicit.als.AlternatingLeastSquares(factors=50,  regularization=0.01, dtype=np.float64, iterations=20, use_gpu=False)
model_song.fit(sparse_song)

In [18]:
repos_song=dict(enumerate(plylst_song_map['songs'].cat.categories))
repo_ids_song={r: i for i, r in repos_song.items()}
A=model_song.similar_items(repo_ids_song['10'])
y=[]
for i in range(len(A)):
  y.insert(i, repos_song[A[i][0]])

print(y)


['10', '2144', '22884', '328456', '328522', '685617', '634084', '159549', '123414', '12913']


In [19]:
plylst_song_map_test=test_song

#test set에 대한 unnesting
plylst_song_map_unnest_test=np.dstack(
    (
        np.repeat(plylst_song_map_test.id.values, list(map(len, plylst_song_map_test.songs))),
        np.concatenate(plylst_song_map_test.songs.values)
    )
)

#unnested 데이터프레임 생성 : plylst_song_map
plylst_song_map_test=pd.DataFrame(data=plylst_song_map_unnest_test[0], columns=plylst_song_map_test.columns)
plylst_song_map_test['id']=plylst_song_map_test['id'].astype('category')
plylst_song_map_test['songs']=plylst_song_map_test['songs'].astype('category')


del plylst_song_map_unnest_test

#test data의 sparse matrix 만들기
test_sparse_song=scipy.sparse.coo_matrix((np.ones(plylst_song_map_test.shape[0]),
                                (plylst_song_map_test['id'].cat.codes.copy(),
                                 plylst_song_map_test['songs'].cat.codes.copy()))).tocsr()


In [20]:
result_song=[]
for i in range(test_song.shape[0]):
  B=model_song.recommend(userid=i, user_items=test_sparse_song, N=100)
  result_B=[]
  for j in range(len(B)):
    B[j]=repos_song[B[j][0]]
    result_B.insert(j, B[j])

  result_song.insert(i, result_B)


In [21]:
test_song['predicted_song']=result_song

In [23]:
test_song.to_csv('테스트 곡 예측')

In [ ]:
song_meta[song_meta['id']==440160]

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
440160,"[GN2502, GN0105, GN2501, GN0101, GN2505]",20110512,비가 오는 날엔,1272951,[436981],비가 오는 날엔,"[GN2500, GN0100]",[비스트],440160


In [ ]:
test_tag=test[['id', 'tags']]
filtering=test_tag['tags'].astype('str')=='[]'
test_tag=test_tag[~filtering]

#플레이리스트 id, tag 추출
plylst_tag_map_train=train[['id', 'tags']]
plylst_tag_map_test=test_tag
plylst_tag_map=pd.concat([plylst_tag_map_train, plylst_tag_map_test])

#unnesting
plylst_tag_map_unnest=np.dstack(
    (
        np.repeat(plylst_tag_map.id.values, list(map(len, plylst_tag_map.tags))),
        np.concatenate(plylst_tag_map.tags.values)
    )
)

plylst_tag_map=pd.DataFrame(data=plylst_tag_map_unnest[0], columns=plylst_tag_map.columns)
plylst_tag_map['id']=plylst_tag_map['id'].astype('category')
plylst_tag_map['tags']=plylst_tag_map['tags'].astype('category')

del plylst_tag_map_unnest

sparse_tag=scipy.sparse.coo_matrix((np.ones(plylst_tag_map.shape[0]),
                                    (plylst_tag_map['tags'].cat.codes.copy(),
                                    plylst_tag_map['id'].cat.codes.copy())))

In [ ]:
model_tag=implicit.als.AlternatingLeastSquares(factors=20,  regularization=0.01, dtype=np.float64, iterations=50, use_gpu=False)
model_tag.fit(sparse_tag)

In [ ]:
repos=dict(enumerate(plylst_tag_map['tags'].cat.categories))
repo_ids={r: i for i, r in repos.items()}

A=model_tag.similar_items(repo_ids['기분전환'])
y=[]
for i in range(len(A)):
  y.insert(i, repos[A[i][0]])

print(y)

['기분전환', '힘이_나는', '입학', '시갈라', '스노보드', '클럽가고싶게만드는', '골든브라더스', '기승전결', '나가볼까요', '흥아솟아라']


In [ ]:
plylst_tag_map_test=test_tag


plylst_tag_map_test_unnest=np.dstack(
    (
        np.repeat(plylst_tag_map_test.id.values, list(map(len, plylst_tag_map_test.tags))),
        np.concatenate(plylst_tag_map_test.tags.values)
    )
)

plylst_tag_map_test=pd.DataFrame(data=plylst_tag_map_test_unnest[0], columns=plylst_tag_map_test.columns)
plylst_tag_map_test['id']=plylst_tag_map_test['id'].astype('category')
plylst_tag_map_test['tags']=plylst_tag_map_test['tags'].astype('category')

del plylst_tag_map_test_unnest

test_sparse_tag=scipy.sparse.coo_matrix((np.ones(plylst_tag_map_test.shape[0]),
                                         (plylst_tag_map_test['id'].cat.codes.copy(),
                                          plylst_tag_map_test['tags'].cat.codes.copy()))).tocsr()


In [ ]:
result_tag=[]
for i in range(test_tag.shape[0]):
  B=model_tag.recommend(userid=i, user_items=test_sparse_tag, N=10)
  result_B=[]
  for j in range(len(B)):
    B[j]=repos[B[j][0]]
    result_B.insert(j, B[j])

  result_tag.insert(i, result_B)



In [ ]:
print(result_tag)

[[138538, ['힙합', '랩', 'HipHop', '국내힙합', '감성힙합', '외힙', '트렌디', '국힙', '외국힙합', '클럽']], [80810, ['카페', '뉴에이지', '겨울', '락', '재즈', '가을', '피아노', '알앤비', '연주곡', '소울']], [136650, ['매장음악', '힐링', '뉴에이지', '휴식', '피아노', '까페', '연주곡', '클래식', '봄', '자장가']], [53131, ['힙합', '랩', 'HipHop', '국내힙합', '감성힙합', '외힙', '트렌디', '국힙', '외국힙합', '클럽']], [2380, ['잔잔한', '가을', '이별', '비오는날', '새벽', '위로', '슬픔', '감성', '휴식', '카페']], [149069, ['카페', '가을', '여행', '알앤비', '휴식', '발라드', '소울', '산책', '비오는날', '봄']], [7373, ['발라드', '가을', '댄스', '어쿠스틱', '산책', '혼자', '명곡', '겨울', '분위기', '아이돌']], [29647, ['휴식', '힐링', '카페', '겨울', '가을', '잔잔한', '까페', '여행', '커피', '신나는']], [105642, ['가을', '카페', '알앤비', '여행', '재즈', '소울', '감성', '휴식', '비오는날', '발라드']], [41619, ['휴식', '힐링', '가을', '이별', '슬픔', '까페', '감성', '위로', '스트레스', '편안한']], [150267, ['힙합', '기분전환', '감성', '랩', '스트레스', 'HipHop', '봄', '트렌디', '위로', '국내힙합']], [65872, ['휴식', '기분전환', '힐링', '스트레스', '위로', '까페', '편안한', '여유', '취향저격', '봄']], [1531, ['뉴에이지', '카페', '힐링', '인디', '피아노', '연주곡', '재즈', '잔잔한', '클래식', '가을']], [2

In [ ]:
test_tag['predicted_tag']=result_tag

In [ ]:
test_tag

,id,tags,predicted_tag
5,138538,"[비, 분위기, 컨트리, 서정적인]","[힙합, 랩, HipHop, 국내힙합, 감성힙합, 외힙, 트렌디, 국힙, 외국힙합,..."
8,80810,[스트레스],"[카페, 뉴에이지, 겨울, 락, 재즈, 가을, 피아노, 알앤비, 연주곡, 소울]"
11,136650,"[아침, 눈, 크리스마스]","[매장음악, 힐링, 뉴에이지, 휴식, 피아노, 까페, 연주곡, 클래식, 봄, 자장가]"
12,53131,[EDM],"[힙합, 랩, HipHop, 국내힙합, 감성힙합, 외힙, 트렌디, 국힙, 외국힙합,..."
17,2380,[생각나],"[잔잔한, 가을, 이별, 비오는날, 새벽, 위로, 슬픔, 감성, 휴식, 카페]"
...,...,...,...
23007,44013,"[휴식, 포크, 커피, 2000년대]","[뉴에이지, 피아노, 연주곡, 힐링, 클래식, 재즈, 자장가, OST, 집중, 공부]"
23008,100684,"[sweetmind, 발라드, 사랑, 기분전환, 역주행]","[사랑, 이별, 설렘, 감성, 슬픔, 인디, 봄, 스트레스, 신나는, 우울]"
23009,17766,[알앤비],"[힐링, 휴식, 뉴에이지, 겨울, 락, 인디, 피아노, 알앤비, 연주곡, 소울]"
23010,101722,[잔잔한],"[기분전환, 잔잔한, 스트레스, 취향저격, 위로, 휴식, 가을, 봄, 센치, 비오는날]"


In [ ]:
test_tag.to_csv('테스트 태그 예측')

KeyError: ignored

'00'

{'': 0,
 '00': 1,
 '007': 2,
 '007시리즈': 3,
 '00s': 4,
 '00년': 5,
 '00년대': 6,
 '00년대노래': 7,
 '00년대발라드': 8,
 '00년도노래': 9,
 '00년생': 10,
 '01': 11,
 '0129': 12,
 '015B': 13,
 '02': 14,
 '03': 15,
 '03월': 16,
 '04': 17,
 '0416': 18,
 '0421': 19,
 '04_29': 20,
 '05': 21,
 '0515': 22,
 '06': 23,
 '07': 24,
 '0802': 25,
 '08월': 26,
 '0930': 27,
 '09월': 28,
 '0channel': 29,
 '0개국어': 30,
 '0살': 31,
 '0시': 32,
 '1': 33,
 '10': 34,
 '100': 35,
 '100000': 36,
 '1000곡': 37,
 '1000만관객동원': 38,
 '1000칼로리': 39,
 '100곡': 40,
 '100번': 41,
 '100선': 42,
 '100억짜리피쳐링': 43,
 '100위': 44,
 '100주년': 45,
 '101': 46,
 '10cm': 47,
 '10곡': 48,
 '10년간의_기록': 49,
 '10년대': 50,
 '10년전': 51,
 '10대': 52,
 '10대가10대에게': 53,
 '10대감성': 54,
 '10대들의우상': 55,
 '10대의영원할것같았던순수함': 56,
 '10만': 57,
 '10만개': 58,
 '10분전': 59,
 '10살': 60,
 '10센치': 61,
 '10월': 62,
 '10월29일': 63,
 '10월4주차': 64,
 '10월_1주차': 65,
 '10월_2주차': 66,
 '10월_3주차': 67,
 '10월_4주차': 68,
 '10월마지막힙합': 69,
 '10월음악': 70,
 '10월힙합': 71,
 '10주': 72,
 '10주년': 73,
 '10주년축하해': 74,

ValueError: ignored

0        2288
1        2250
2        4186
3        2485
4        2700
         ... 
27333    3007
27334    4838
27335    2885
27336     784
27337    3396
Length: 27338, dtype: int16

0              비
1            분위기
2            컨트리
3           서정적인
4           스트레스
          ...   
27333        어머니
27334        힘들때
27335         아빠
27336         가족
27337    위로받고싶을때
Name: tags, Length: 27338, dtype: category
Categories (4875, object): [007, 00년대, 0살, 10, ..., 힙합재즈, 힙합존, 힙합클럽, 힙합홍대]

In [ ]:
repos=dict(enumerate(plylst_tag_map['tags'].cat.categories))
repo_ids={r: i for i, r in repos.items()}
model_tag.similar_items(repo_ids['007'])


[(2, 0.0006527478101868229),
 (17148, 0.0006526129768950573),
 (12978, 0.0006526129768950573),
 (18870, 0.0006526129768950573),
 (14090, 0.0006526129768950573),
 (28921, 0.0006448860736241609),
 (7617, 0.0006341237661496249),
 (12416, 0.0006341237661496246),
 (17379, 0.0006339816717712471),
 (13786, 0.0006339816717712471)]

In [ ]:
def user_items(u_stars):
  star_ids=[repo_ids[s] for s in u_stars if s in repo_ids]
  data=[1 for _ in star_ids]
  rows = [0 for _ in star_ids]
  shape=(1, model_tag.item_factors.shape[0])
  return scipy.sparse.coo_matrix((data, (rows, star_ids)), shape=shape).tocsr()

A=user_items('기분전환')

def recommend(user_item):
  recs = model_tag.recommend(userid=0, user_items=user_item, recalculate_user=False)
  return [(repos[r], s) for r, s in recs]

recommend(A)

[('랩', 0.9985795467584847),
 ('힙합', 0.9941519333826693),
 ('Rap', 0.033352881349581635),
 ('HipHop', 0.03319703872596241),
 ('국내힙합', 0.026553907089736963),
 ('외힙', 0.019664926941388203),
 ('외국힙합', 0.01852408984848225),
 ('국힙', 0.017887751680423215),
 ('비트', 0.016760270539377432),
 ('소울', 0.015366553958381768)]

In [ ]:
model_tag.recommend(userid=0, user_items=A)

[(10976, 0.9985795467584847),
 (30120, 0.9941519333826693),
 (2916, 0.033352881349581635),
 (2063, 0.03319703872596241),
 (6742, 0.026553907089736963),
 (20809, 0.019664926941388203),
 (20754, 0.01852408984848225),
 (6784, 0.017887751680423215),
 (14858, 0.016760270539377432),
 (16364, 0.015366553958381768)]

In [ ]:
model_tag.recommend(userid=0, user_items=A)

[(10976, 0.9985795467584847),
 (30120, 0.9941519333826693),
 (2916, 0.033352881349581635),
 (2063, 0.03319703872596241),
 (6742, 0.026553907089736963),
 (20809, 0.019664926941388203),
 (20754, 0.01852408984848225),
 (6784, 0.017887751680423215),
 (14858, 0.016760270539377432),
 (16364, 0.015366553958381768)]

[('랩', 0.9985795467584847),
 ('힙합', 0.9941519333826693),
 ('Rap', 0.033352881349581635),
 ('HipHop', 0.03319703872596241),
 ('국내힙합', 0.026553907089736963),
 ('외힙', 0.019664926941388203),
 ('외국힙합', 0.01852408984848225),
 ('국힙', 0.017887751680423215),
 ('비트', 0.016760270539377432),
 ('소울', 0.015366553958381768)]